In [94]:


import os
import re
import string
import glob
import nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from nltk.corpus import stopwords
from collections import Counter
from nltk import word_tokenize

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score



In [95]:
text_dir = "../data/raw/text"
labels_dir =  "../data/raw/labels"
audio_dir = "../data/raw/audio"



In [96]:
'_7HVhnSYX1Y___21___125.556___127.594___Nothing dangerous'.split('___')[1]

'21'

In [97]:
text_files = glob.glob(f"{text_dir}/*.txt")


def remove_stamps_str(line)->str:
    #clip_num = re.search('.+___\d\d?\d?___.+', line).group(0)
    stamp = re.search('.+___', line).group(0)
    new_line = line.strip(stamp)
    return new_line

In [98]:

## Pour python < 3.9, sinon str.removeprefix() de base
def removeprefix(self: str, prefix: str, /) -> str:
    if self.startswith(prefix):
        return self[len(prefix):]
    else:
        return self[:]



#Retire chaque stamps, chaque texte devien 1 seul str

def text_list_generator(files_list):
    text_list = []
    for filename in files_list:
        with open(file = filename, encoding = 'utf-8') as f:

            ##WINDOWS SPECIFIC
           # videoid = filename.removeprefix(text_dir + '\\').rstrip('.txt')
            videoid = removeprefix(filename, text_dir + '\\').rstrip('.txt')
            lines = f.readlines()
            for line_number, text_line in enumerate(lines):
                clean_line = remove_stamps_str(text_line)
                clip_id = videoid +'_'+ text_line.split('___')[1]
                #clip_id = videoid +'_' +str(line_number)
                yield (clip_id, clean_line.rstrip())

"""
corpus = []
text = []
text_list = []
for filename in text_files:
    text_str = ''
    with open(file=filename, encoding='utf-8') as f:
        lines = f.readlines()
        for count, line in enumerate(lines):
            clean_line = remove_stamps_str(line)
            text.append(clean_line)
    text_list.append(text)
    """

"\ncorpus = []\ntext = []\ntext_list = []\nfor filename in text_files:\n    text_str = ''\n    with open(file=filename, encoding='utf-8') as f:\n        lines = f.readlines()\n        for count, line in enumerate(lines):\n            clean_line = remove_stamps_str(line)\n            text.append(clean_line)\n    text_list.append(text)\n    "

In [99]:
stop_words = stopwords.words('English')

#création d'un dict pour lookup en O(1)
stopwords_dict = Counter(stop_words)


#Retire tous les timestamps en début de ligne, présents dans chaque transcript
def remove_stamps_str(line)->str:
    stamp = re.search('.+___', line).group(0)
    new_line = line.strip(stamp)
    return new_line

#Retire les charactères non-ascii 
def remove_nonascii(line)->str:
    ascii_line = line.encode(encoding = 'ascii', errors = 'ignore').decode()
    return ascii_line

#met tout en minuscules, retire les nombres et stopwords
def clean_stopwords_digits(line)->str:
    new_line = line.translate(str.maketrans('', '', string.punctuation))
    new_line = ' '.join([word.lower() for word in new_line.split() if (len(word) >=2 and word.isalpha() and word not in stopwords_dict)])
    return new_line



In [100]:
label_df = pd.read_csv("../data/interim/labels/labels.csv")
display(label_df)

,id,anger,disgust,fear,happiness,sadness,surprise,sentiment
0,--qXJuDtHPw_5,0,0,0,1,0,0,1
1,-3g5yACwYnA_10,0,0,0,0,0,0,1
2,-3g5yACwYnA_13,0,0,0,0,0,0,1
3,-3g5yACwYnA_2,0,0,0,1,0,0,0
4,-3g5yACwYnA_3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
21813,zwTrXwi54us_6,0,0,0,0,0,0,0
21814,zwTrXwi54us_7,0,0,0,0,0,0,0
21815,zwTrXwi54us_8,0,0,0,0,0,0,0
21816,zwTrXwi54us_9,0,0,0,0,0,0,1


In [101]:
corpus = (text for text in text_list_generator(text_files))
df_text = pd.DataFrame(corpus)
df_text.columns = ['id', 'text']

In [102]:
display(df_text.head(5))

,id,text
0,--qXJuDtHPw_0,I see that there are three category of writers.
1,--qXJuDtHPw_1,"I define them as being an author, a writer, an..."
2,--qXJuDtHPw_2,"An author, I like to classify as somebody who ..."
3,--qXJuDtHPw_3,These are the well-known authors of our time a...
4,--qXJuDtHPw_4,"Then, there is the writer."


In [103]:

df_text['text'] =  df_text.text.apply(remove_nonascii)


df_text['clean_text'] = df_text.text.apply(lambda s : clean_stopwords_digits(s))
df_text['tokens'] = df_text.clean_text.apply(lambda x : x.split(' '))

#df_text['lines'] = df_text.groupby('unique_id')
df_text['count'] = df_text['text'].apply(lambda x : len(x.split(' ')))




display(df_text)

,id,text,clean_text,tokens,count
0,--qXJuDtHPw_0,I see that there are three category of writers.,see three category writers,"[see, three, category, writers]",9
1,--qXJuDtHPw_1,"I define them as being an author, a writer, an...",define author writer story teller,"[define, author, writer, story, teller]",13
2,--qXJuDtHPw_2,"An author, I like to classify as somebody who ...",an author like classify somebody writes great ...,"[an, author, like, classify, somebody, writes,...",14
3,--qXJuDtHPw_3,These are the well-known authors of our time a...,these wellknown authors time past,"[these, wellknown, authors, time, past]",11
4,--qXJuDtHPw_4,"Then, there is the writer.",then writer,"[then, writer]",5
...,...,...,...,...,...
44972,_ZlF5Q9W1DM_1,Clearly because everything is data driven toda...,clearly everything data driven today measurabl...,"[clearly, everything, data, driven, today, mea...",17
44973,_ZlF5Q9W1DM_2,"But the conceptual drivers, what sort of drive...",but conceptual drivers sort drives passions th...,"[but, conceptual, drivers, sort, drives, passi...",28
44974,_ZlF5Q9W1DM_3,So those two things actually I think rub up ag...,so two things actually think rub collide reall...,"[so, two, things, actually, think, rub, collid...",21
44975,_ZlF5Q9W1DM_4,So the idea of numbers telling the story up ag...,so idea numbers telling story idea explaining ...,"[so, idea, numbers, telling, story, idea, expl...",40


In [104]:
#frame = frame[~frame[target_features].sum(axis = 1) == 0]

In [105]:
frame = pd.merge(df_text, label_df, on = 'id', how = 'inner')
frame

,id,text,clean_text,tokens,count,anger,disgust,fear,happiness,sadness,surprise,sentiment
0,--qXJuDtHPw_5,I see that a writer is somebody who has an inc...,see writer somebody incredible command mechani...,"[see, writer, somebody, incredible, command, m...",18,0,0,0,1,0,0,1
1,-3g5yACwYnA_2,Key Polymer brings a technical aspect to our o...,key polymer brings technical aspect operation ...,"[key, polymer, brings, technical, aspect, oper...",14,0,0,0,1,0,0,0
2,-3g5yACwYnA_3,We're a huge user of adhesives for our operati...,were huge user adhesives operation called floc...,"[were, huge, user, adhesives, operation, calle...",29,0,0,0,0,0,0,0
3,-3g5yACwYnA_4,Key brings those types of aspects to a busines...,key brings types aspects business new markets ...,"[key, brings, types, aspects, business, new, m...",34,0,0,0,0,0,0,0
4,-3g5yACwYnA_9,We have many new opportunities through the way...,we many new opportunities way things changed y...,"[we, many, new, opportunities, way, things, ch...",21,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
21593,_xjBqNfiEY4_6,"And once again, U of I students are so well pr...",and students well prepared asset student,"[and, students, well, prepared, asset, student]",24,0,0,0,0,0,0,0
21594,_XWUKMkxa-Q_4,So its become more of an iterative process wit...,so become iterative process fast results fast ...,"[so, become, iterative, process, fast, results...",13,0,0,0,1,0,0,1
21595,_YzIvwvyIq4_5,"Secondly, using social, you know, things like ...",secondly using social know things like twitter...,"[secondly, using, social, know, things, like, ...",30,0,0,0,1,0,0,1
21596,_ZlF5Q9W1DM_0,JOHN GERZEMA: When I think about finding insig...,john gerzema when think finding insights consu...,"[john, gerzema, when, think, finding, insights...",24,0,0,0,0,0,0,0


In [109]:
emotions = ['anger', 'disgust', 'fear','happiness', 'sadness','surprise']

In [115]:
frame = frame[frame[emotions].sum(axis=1) >0]

In [116]:
#nltk.download('averaged_perceptron_tagger')
#df_text['tagged'] = df_text['tokens'].apply(nltk.tag.pos_tag )

KeyboardInterrupt: 

In [ ]:
target_features = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'sentiment']

#frame['target_vector'] = frame[target_features].apply(list, axis=1)

Bag of words

In [117]:
count_vector = CountVectorizer(lowercase=False, ngram_range=(1,3), max_features=250)
counts = count_vector.fit_transform(frame.clean_text)


In [118]:
X_train, X_test, y_train, y_test = tts(frame.clean_text,frame[target_features], test_size=0.2)

bow_data = tts(frame.clean_text, frame[target_features], test_size = 0.2)

In [119]:
nb_pipeline = Pipeline([
    ('BoW', count_vector),
    ('classifier', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior = None)))
])

In [120]:
nb_pipeline.fit(X_train, y_train)
preds = nb_pipeline.predict(X_test)

print(f'Test accuracy : {accuracy_score(y_test, preds).round(3)}')
preds

ValueError: Multioutput target data is not supported with label binarization

In [121]:
for feature in target_features:
    print(f'Fitting for {feature}')
    nb_pipeline.fit(X_train, y_train[feature])
    preds = nb_pipeline.predict(X_test)
    print(f'Test accuracy : {accuracy_score(y_test[feature], preds)}')

Fitting for anger
Test accuracy : 0.8743604320636725
Fitting for disgust
Test accuracy : 0.8567367822626493
Fitting for fear
Test accuracy : 0.9874928936895964
Fitting for happiness
Test accuracy : 0.7674815235929505
Fitting for sadness
Test accuracy : 0.8544627629334849
Fitting for surprise
Test accuracy : 0.9778283115406481
Fitting for sentiment
Test accuracy : 0.602615122228539


In [ ]:
import gensim.downloader as api
from gensim.models import Word2Vec as w2v

#w2v_model = api.load("word2vec-google-news-300")


#SEULEMENT PRENDRE MOT DANS MODEL.VOCAB